## 1. Load and encode datasets with the AgingPPEncoder

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Load dataset, dropping id column
df = pd.read_csv('train.csv').iloc[0:,1:]
original_cols = df.columns
df.columns = range(len(df.columns))

# Set which variables are categorical
cat_cols = df.columns[:-1]

# Print summary of df
print(df.shape)
df.head()

(300000, 24)


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [2]:
# Load encoded dataset if already exists or encode directly
import os.path
import time

if os.path.isfile('encoded_Aging.csv'):
    encoded = pd.read_csv('encoded_Aging.csv', header=None)
else:
    # Import encoder
    import sys
    sys.path.append('../encoders')
    from pattern_preserving import AgingPPEncoder
    aging = AgingPPEncoder(cycles=400, n_thread=4)
    
    # Prepare sample to fit encoder
    sample = df.sample(frac=0.3, replace=False).reset_index(drop=True)
    X = sample[sample.columns[:-1]]
    y = sample[sample.columns[-1]]
    
    # Fit encoder
    tic = time.perf_counter()
    aging.fit(X, y, cat_cols)
    toc = time.perf_counter()
    print('Fitting completed in', round(toc-tic,1), 'seconds')
    
    # Encode whole dataset, and save for future use
    tic = time.perf_counter()
    encoded = aging.transform(df[df.columns[:-1]])
    toc = time.perf_counter()
    print('Transformed full dataset in', round(toc-tic,1), 'seconds')
    encoded[encoded.shape[1]] = df[df.columns[-1]]
    encoded.to_csv('encoded_Aging.csv', index=None, header=None)
    
    # Encode test dataset
    test = pd.read_csv('test.csv', low_memory=False)
    test.columns = range(test.shape[1])
    test = test.loc[0:,1:]    
    test.columns = range(test.shape[1])
    tic = time.perf_counter()
    test = aging.transform(test)
    toc = time.perf_counter()
    print('Transformed test dataset in', round(toc-tic,1), 'seconds')
    test.to_csv('test_Aging.csv', index=None, header=None)
    
encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.297559,0.746739,0.260123,0.011265,0.437122,0.750276,0.956398,0.173252,0.516330,0.169952,...,0.413516,0.834499,0.235358,0.000823,0.069315,0.380969,0.483055,0.626921,0.585144,0
1,0.297559,0.387710,0.260123,0.011265,0.437122,0.750276,0.396622,0.559237,0.382145,0.393306,...,0.628408,0.206559,0.235358,0.804297,0.134973,0.366559,0.913697,0.445777,0.069831,0
2,0.297559,0.746739,0.260123,0.202940,0.437122,0.485763,0.396622,0.440309,0.382145,0.718073,...,0.432455,0.206559,0.043688,0.140372,0.069315,0.529304,0.529973,0.445777,0.585144,0
3,0.297559,0.387710,0.260123,0.202940,0.437122,0.227766,0.396622,0.173252,0.504937,0.815834,...,0.566422,0.206559,0.235358,0.361620,0.635701,0.380969,0.603184,0.626921,0.225614,1
4,0.297559,0.746739,0.260123,0.202940,0.681525,0.227766,0.396622,0.440309,0.504937,0.815834,...,0.269755,0.206559,0.235358,0.359491,0.134973,0.529304,0.669337,0.445777,0.069831,0


## 2. Search for best xgboost parameters with Hyperopt

In [3]:
# Define objective function for Hyperopt (credit to Leonardo Ferreira, 
# https://www.kaggle.com/kabure/eda-feat-engineering-encode-conquer)
from sklearn.model_selection import KFold
import xgboost as xgb
import time
from hyperopt import hp, fmin, tpe, space_eval

def objective(params):
    tic = time.perf_counter()
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'subsample': "{:.2f}".format(params['subsample']),
        'reg_alpha': "{:.3f}".format(params['reg_alpha']),
        'reg_lambda': "{:.3f}".format(params['reg_lambda']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction'])
    }

    print("\n>> Hyperopt New Run")
    print(f"params = {params}")
    FOLDS = 8
    count=1
    max_rounds = 500
    kf = KFold(n_splits=FOLDS, shuffle=False)
    
    mean_auc = 0.0
    for train_idx, val_idx in kf.split(X, y):
        # Prepare data for xgboost
        dtrain = xgb.DMatrix(X.iloc[train_idx, :], label=y.iloc[train_idx])
        dval = xgb.DMatrix(X.iloc[val_idx, :], label=y.iloc[val_idx])
        evallist = [(dtrain, 'train'), (dval, 'eval')]
        
        params['objective']= 'binary:logistic'
        params['eval_metric']= 'auc'
        params['n_thread'] = 7
        evals_result={}
        bst = xgb.train(params, dtrain, max_rounds, evallist, 
                        early_stopping_rounds=30, verbose_eval=False,
                        evals_result=evals_result) 
        
        auc = evals_result['eval']['auc'][-1]
        tauc = evals_result['train']['auc'][-1]
        mean_auc += auc/FOLDS
        print(f'Fold {count} - train_auc {round(tauc, 5)} - val_auc: {round(auc, 5)}')
        count += 1
        del dtrain, dval, bst
    toc = time.perf_counter()
    print('Run time:', round(toc-tic, 1), 'seconds.')
    print('Mean val_auc:', round(mean_auc,5))
    
    return 1-mean_auc

# Define search space for Hyperopt (based on the one by Leonardo Ferreira, 
# https://www.kaggle.com/kabure/eda-feat-engineering-encode-conquer)
space = {
    'max_depth': hp.choice('max_depth', [3, 4, 5, 6, 7]),
    'reg_alpha':  hp.uniform('reg_alpha', 0.05, 0.4),
    'reg_lambda': hp.uniform('reg_lambda', 0.05, 0.4),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.15),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 0.8),
    'gamma': hp.uniform('gamma', 0.01, 0.7),
    'subsample': hp.choice('subsample', [0.5, 0.6, 0.7, 0.8]),
    'feature_fraction': hp.uniform('feature_fraction', 0.4, 0.8),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, 0.9) }

In [4]:
# Take sample for xgboost parameter optimisation
sample = encoded.sample(frac=1, replace=False)
X = sample[sample.columns[:-1]]
y = sample[sample.columns[-1]]

# Run hyperopt 
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=15)

# Set best params
best_params = space_eval(space, best)
best_params 

                                                      
>> Hyperopt New Run
params = {'max_depth': 7, 'gamma': '0.436', 'subsample': '0.60', 'reg_alpha': '0.195', 'reg_lambda': '0.186', 'learning_rate': '0.135', 'colsample_bytree': '0.403', 'feature_fraction': '0.483', 'bagging_fraction': '0.496'}
Fold 1 - train_auc 0.88013 - val_auc: 0.77015         
Fold 2 - train_auc 0.88068 - val_auc: 0.76771         
Fold 3 - train_auc 0.86056 - val_auc: 0.77097         
Fold 4 - train_auc 0.85444 - val_auc: 0.7686          
Fold 5 - train_auc 0.87747 - val_auc: 0.77099         
Fold 6 - train_auc 0.8685 - val_auc: 0.7701           
Fold 7 - train_auc 0.85294 - val_auc: 0.77166         
Fold 8 - train_auc 0.86287 - val_auc: 0.77475         
Run time:                                             
382.7                                                 
seconds.                                              
Mean val_auc:                                         
0.77062                                   

{'bagging_fraction': 0.49506046299739476,
 'colsample_bytree': 0.6297590974801108,
 'feature_fraction': 0.4492564451220282,
 'gamma': 0.6477207672507288,
 'learning_rate': 0.11217192884480312,
 'max_depth': 4,
 'reg_alpha': 0.14022035342751765,
 'reg_lambda': 0.1676888394121102,
 'subsample': 0.8}

## Train best model further and make predictions to submit

In [5]:
# Train xgboost model further with best params and full dataset
from sklearn.model_selection import train_test_split
train, val = train_test_split(encoded, train_size=0.75, random_state=0)
dtrain = xgb.DMatrix(train[train.columns[:-1]], label=train[train.columns[-1]])
dval = xgb.DMatrix(val[val.columns[:-1]], label=val[val.columns[-1]])

evallist = [(dtrain, 'train'), (dval, 'eval')]
evals_result = {}
best_params['objective']= 'binary:logistic'
best_params['eval_metric']= 'auc'
best_params['n_thread'] = 7
bst = xgb.train(best_params, dtrain, 4000, evallist, early_stopping_rounds=50, 
                verbose_eval=False, evals_result=evals_result) 

print('Train auc '+str(evals_result['train']['auc'][-1])+', Test auc '+str(evals_result['eval']['auc'][-1]))

Train auc 0.847602, Test auc 0.780741


In [6]:
# Load test set
test = pd.read_csv('test_Aging.csv', header=None, low_memory=False)
index = pd.read_csv('test.csv', low_memory=False)['id'].values
dtest = xgb.DMatrix(test)
# Make predictions and save
ypred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

if not os.path.exists('submissions'):
    os.mkdir('submissions')
pd.DataFrame({'id':index, 'target':ypred}, index=None).to_csv('./submissions/predicted_Aging.csv', index=None)